In [6]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.cm as cm
import pandas as pd

AttributeError: partially initialized module 'pandas' from '/Users/sparsh/Documents/MachineLearning/venv/lib/python3.13/site-packages/pandas/__init__.py' has no attribute '_pandas_datetime_CAPI' (most likely due to a circular import)

In [4]:
# Load the dataset
try:
    # The palmerpenguins package provides a clean version. We load it from its source.
    url = 'https://raw.githubusercontent.com/allisonhorst/palmerpenguins/main/inst/extdata/penguins.csv'
    df = pd.read_csv(url)
    print("Palmer Penguins dataset loaded successfully.")
except Exception as e:
    print(f"Failed to load dataset: {e}")
    df = pd.DataFrame() # Create empty dataframe to avoid further errors

Failed to load dataset: name 'pd' is not defined


NameError: name 'pd' is not defined

In [ ]:
if not df.empty:
    print("\nOriginal Dataset Info:")
    df.info()

    # Handle missing values by dropping them (there are only a few)
    df_cleaned = df.dropna()
    print(f"\nDataset shape after dropping NA: {df_cleaned.shape}")

    # For clustering, we will use the numerical features
    numerical_features = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
    X = df_cleaned[numerical_features]

    # Keep the true species labels for later evaluation
    y_true = df_cleaned['species']

    print("\nSelected features for clustering:")
    print(X.head())

In [ ]:
if not df.empty:
    print("\nOriginal Dataset Info:")
    df.info()

    # Handle missing values by dropping them (there are only a few)
    df_cleaned = df.dropna()
    print(f"\nDataset shape after dropping NA: {df_cleaned.shape}")

    # For clustering, we will use the numerical features
    numerical_features = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
    X = df_cleaned[numerical_features]

    # Keep the true species labels for later evaluation
    y_true = df_cleaned['species']

    print("\nSelected features for clustering:")
    print(X.head())

In [ ]:
if 'X' in locals():
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    print("\nData has been scaled using StandardScaler.")
    print("First 5 rows of scaled data:")
    print(pd.DataFrame(X_scaled, columns=numerical_features).head())

In [ ]:
if 'X_scaled' in locals():
    wcss = []
    silhouette_scores = []
    k_range = range(2, 11) # Start from 2 for silhouette score

    for k in k_range:
        kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42, n_init=10)
        kmeans.fit(X_scaled)
        wcss.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

    # Plotting
    plt.figure(figsize=(15, 6))

    # Elbow Method Plot
    plt.subplot(1, 2, 1)
    plt.plot(k_range, wcss, marker='o', linestyle='--')
    plt.title('Elbow Method')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('WCSS')
    plt.grid(True)

    # Average Silhouette Score Plot
    plt.subplot(1, 2, 2)
    plt.plot(k_range, silhouette_scores, marker='o', linestyle='--', color='r')
    plt.title('Average Silhouette Score vs. k')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Average Silhouette Score')
    plt.grid(True)

    plt.tight_layout()
    plt.show()
    # The silhouette score plot peaks at k=2, but k=3 (the actual number of species)
    # also has a high score and a good elbow. Let's explore these visually.

In [ ]:
if 'X_scaled' in locals():
    for k in [2, 3, 4]:
        # Create a subplot with 1 row and 1 column
        fig, ax1 = plt.subplots(1, 1)
        fig.set_size_inches(10, 7)

        # The 1st subplot is the silhouette plot
        ax1.set_xlim([-0.2, 1])
        # The (k+1)*10 is for inserting blank space between silhouette plots of individual clusters
        ax1.set_ylim([0, len(X_scaled) + (k + 1) * 10])

        # Initialize the clusterer with k value
        clusterer = KMeans(n_clusters=k, init='k-means++', random_state=42, n_init=10)
        cluster_labels = clusterer.fit_predict(X_scaled)

        # The silhouette_score gives the average value for all the samples.
        silhouette_avg = silhouette_score(X_scaled, cluster_labels)
        print(f"For n_clusters = {k}, the average silhouette_score is : {silhouette_avg:.4f}")

        # Compute the silhouette scores for each sample
        sample_silhouette_values = silhouette_samples(X_scaled, cluster_labels)

        y_lower = 10
        for i in range(k):
            # Aggregate the silhouette scores for samples belonging to cluster i, and sort them
            ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
            ith_cluster_silhouette_values.sort()
            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i
            color = cm.nipy_spectral(float(i) / k)
            ax1.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title(f"Silhouette plot for K = {k}")
        ax1.set_xlabel("Silhouette coefficient values")
        ax1.set_ylabel("Cluster label")
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--") # The average silhouette score
        ax1.set_yticks([])
        ax1.set_xticks(np.arange(-0.2, 1.1, 0.2))

    plt.show()
    # Observation: The plot for K=3 shows clusters of more uniform thickness and positive scores,
    # suggesting it's a better fit than K=2 or K=4. We will choose k=3.
    print("\nBased on the analysis, the optimal number of clusters is 3.")

In [ ]:
if 'X_scaled' in locals():
    optimal_k = 3
    kmeans = KMeans(n_clusters=optimal_k, init='k-means++', random_state=42, n_init=10)
    y_kmeans = kmeans.fit_predict(X_scaled)

    # Add cluster info to the cleaned dataframe
    df_cleaned['cluster'] = y_kmeans

    # Reduce dimensionality with PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    df_pca = pd.DataFrame(data=X_pca, columns=['PCA1', 'PCA2'])
    df_pca['cluster'] = y_kmeans
    df_pca['species'] = y_true.values

    print(f"\nFinal K-Means model trained with k={optimal_k}.")
    print(f"Data reduced to 2 Principal Components. Explained variance ratio: {pca.explained_variance_ratio_}")



In [ ]:
if 'df_pca' in locals():
    plt.figure(figsize=(16, 7))

    # Plot 1: Colored by K-Means Cluster
    plt.subplot(1, 2, 1)
    sns.scatterplot(x='PCA1', y='PCA2', hue='cluster', data=df_pca, palette='viridis', s=100, alpha=0.8)
    # Plot centroids
    centroids_pca = pca.transform(kmeans.cluster_centers_)
    plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1], s=300, c='red', marker='*', label='Centroids')
    plt.title('Penguin Clusters by K-Means (PCA)')
    plt.legend()
    plt.grid(True)

    # Plot 2: Colored by True Species
    plt.subplot(1, 2, 2)
    sns.scatterplot(x='PCA1', y='PCA2', hue='species', data=df_pca, palette='deep', s=100, alpha=0.8)
    plt.title('True Penguin Species (PCA)')
    plt.grid(True)

    plt.show()
